In [1]:
#!pip install datasets transformers
#!pip install sentencepiece
#!pip install sklearn
#!pip install numpy
#!pip install keras-adamw
#!pip install tensorflow-addons
#!pip install helasentilex

     |████████████████████████████████| 264 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 87.9 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 679 kB/s s eta 0:00:01
     |████████████████████████████████| 23.6 MB 139.0 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 723 kB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 810 kB/s  eta 0:00:01
     |████████████████████████████████| 119 kB 90.1 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 118.0 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 152.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 86.0 MB/s eta 0:00:01
     |████████████████████████████████| 761 kB 100.8 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 39.8 MB/s eta 0:00:01
     |████████████████████████████████| 662 kB 154.9 MB/s eta 0:00:01
     |████████████████████████

     |████████████████████████████████| 65 kB 385 kB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 26.3 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 65.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 38.0 MB/s eta 0:00:01
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.36.1
    Uninstalling grpcio-1.36.1:
      Successfully uninstalled grpcio-1.36.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  A

     |████████████████████████████████| 65 kB 292 kB/s eta 0:00:011


In [2]:
import collections
import pickle
import re
import random
import sys
import os 
import time
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,precision_recall_fscore_support

import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import cumsum

from transformers import pipeline

import tensorflow as tf
from tensorflow import keras



2022-01-10 12:17:54.113820: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#import os; os.environ["TF_KERAS"]='1'
#from keras_adamw import AdamW

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)
tf.test.is_gpu_available()

Num GPUs Available:  1
2.4.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2021-09-03 17:34:16.037247: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-03 17:34:16.037780: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-03 17:34:16.092042: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-03 17:34:16.093026: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2021-09-03 17:34:16.093112: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-09-03 17:34:16.095155: I tensorflow/stream_executor/platform/default/ds

True

2021-09-03 17:34:16.110732: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-03 17:34:16.112321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2021-09-03 17:34:16.112394: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-09-03 17:34:16.112443: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-09-03 17:34:16.112488: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2021-09-03 17:34:16.112534: I tensorflow/stream_executor/platfo

In [2]:
domaindata=pd.read_csv('Data/domaincomments.csv')
domaindata=domaindata.sample(frac=1)

In [2]:
model_checkpoint = "jplu/tf-xlm-roberta-base"##xlm-roberta-base #our model here
batch_size = 16

In [3]:
model_checkpoint = "jplu/tf-xlm-roberta-large"##xlm-roberta-base #our model here
batch_size = 8

In [17]:
folder_path = 'Data/'

lankadeepa_data_path = folder_path + 'lankadeepa_tagged_comments.csv'
gossip_lanka_data_path = folder_path + 'gossip_lanka_tagged_comments.csv'
lankadeepa_data = pd.read_csv(lankadeepa_data_path)[:9059]#just the end of document
gossipLanka_data = pd.read_csv(gossip_lanka_data_path)
gossipLanka_data = gossipLanka_data.drop(columns=['Unnamed: 3'])

all_data = pd.concat([lankadeepa_data,gossipLanka_data], ignore_index=True)
#all_data=all_data[all_data['label']!=5]
#all_data.shape

#small_data=all_data[1000:]
#small_data2=all_data[:1000]
#small_data2=small_data2.reset_index(drop=True)

#bal_data_2=small_data.loc[small_data['label']==2].sample(n=2000,random_state=42)#1500
#bal_data_3=small_data.loc[small_data['label']==3].sample(n=1966,random_state=42)#
#bal_data_4=small_data.loc[small_data['label']==4].sample(n=2000,random_state=42)#
#bal_data_5=small_data.loc[small_data['label']==5].sample(n=1890,random_state=42)#

#bal_data=pd.concat([bal_data_2,bal_data_3,bal_data_4,bal_data_5],ignore_index=True)#
#bal_data=bal_data.sample(frac=1)
#small_data=small_data.reset_index(drop=True)
#bal_data

In [3]:
all_data=pd.read_csv('Data/newssource2.csv').sample(frac=1)#news source heads


In [4]:
def text_preprocessing(data):
    comments=[]
    labels=[]
    comments = list(data['comment'])
    labels = list(data['label'])
   
    comments_splitted = []
    #labels_rec=labels
    #for comment in comments:
    #    lines = []
    #    try:
    #        words = comment.split()
    #        lines += words
    #    except:
    #        continue
    #    comments_splitted.append(lines)
    for i in range(len(labels)): 
        if labels[i]==2:#n
            labels[i]=0
            #print('0')
        elif labels[i]==3:#neu
            labels[i]=1##################
            #print('3')
        elif labels[i]==4:#p,(neu)
            labels[i]=2
            #print('2')
        elif labels[i]==5:#con
            labels[i]=3
            #print('1')
  
  #labels should be starting from 0
    return comments,labels

def text_preprocessing2(data):
    comments=[]
    labels=[]
    comments = list(data['comments'])
    labels = list(data['labels'])#labels
    
    return comments,labels

def text_preprocessing3(data):
    comments=[]
    labels=[]
    comments = list(data['heads'])
    labels = list(data['labels'])#labels
    
    return comments,labels

def text_preprocessing5(data):
    comments=[]
    labels=[]
    comments = list(data['comment'])
    labels = list(data['label'])
   
   
    for i in range(len(labels)): 
        if labels[i]=='negative':#n
            labels[i]=0
            #print('0')
        elif labels[i]=='neutral':#p
            labels[i]=1
            #print('3')
        elif labels[i]=='positive':#neu
            labels[i]=2
            #print('2')
       
    return comments,labels

def text_preprocessing4(data):
    comments=[]
    labels=[]
    comments = list(data['Tweets'])
    labels = list(data['Emotion Set'])
   
    comments_splitted = []
    #labels_rec=labels
    #for comment in comments:
    #    lines = []
    #    try:
    #        words = comment.split()
    #        lines += words
    #    except:
    #        continue
    #    comments_splitted.append(lines)
    for i in range(len(labels)): 
        if labels[i]=='Sadness':#n
            labels[i]=0
            #print('0')
        elif labels[i]=='Anger':#p
            labels[i]=1
            #print('1')
        elif labels[i]=='Fear':#neu
            labels[i]=2
            #print('2')
        elif labels[i]=='Disgust':#con
            labels[i]=3
            #print('3')
        elif labels[i]=='Surprise':#con
            labels[i]=4
            #print('3')
        elif labels[i]=='Happy':#con
            labels[i]=5
            #print('3')
            
  
  #labels should be starting from 0
    return comments,labels
    
def text_preprocessing8(data):
    comments=[]
    labels=[]
    comments = list(data['comments'])
    labels = list(data['labels'])
   
   
    for i in range(len(labels)): 
        if labels[i]=='BLOG':#n
            labels[i]=0
            #print('0')
        elif labels[i]=='CREATIVE':#p
            labels[i]=1
            #print('3')
        elif labels[i]=='ACADEMIC':#neu
            labels[i]=2            #print('2')
        elif labels[i]=='NEWS':#neu
            labels[i]=3
            #print('2')
       
    return comments,labels

In [5]:
ws='Data/writesty3.csv'
all_data=pd.read_csv(ws)
#all_data=all_data[all_data['labels']!='CREATIVE']

#comments,labels=text_preprocessing8(all_data)

In [5]:
def dzsize(n,data):
    ws=['ACADEMIC','NEWS','CREATIVE','BLOG']
    dz=pd.DataFrame()
    for i in ws:#range(6):
        f=int(np.floor(n/4))
        dz=dz.append(((all_data[all_data['labels']==i]).sample(frac=1)).iloc[:f])
        #print(i)
    
    return dz


        

In [5]:
dz1=dzsize(11000,all_data)
dz1

NameError: name 'dzsize' is not defined

In [6]:
#from sklearn import preprocessing
#comment_texts, comment_labels = text_preprocessing(small_data)
##arr =comment_labels.values
##comment_texts, comment_labels = text_preprocessing(bal_data)
#comment_texts2, comment_labels2 = text_preprocessing(small_data2)

comment_texts_all, comment_labels_all=text_preprocessing8(all_data)#(d)#
#comment_texts_all2, comment_labels_all2=text_preprocessing5(all_data2)
#comment_texts_all, comment_labels_all=text_preprocessing(dz)
#comment_texts_all2,comment_labels_all2=text_preprocessing(sentedit[1506:11506])


#comment_texts_all=comment_texts_all+comment_texts_all2
#comment_labels_all=comment_labels_all+comment_labels_all2
#comment_texts_all, comment_labels_all=text_preprocessing2(domaindata)

#comment_texts_all, comment_labels_all=text_preprocessing4(emotiondata)


# we then initialize the zero array
#labels = np.zeros((len(small_data), arr.max()+1))

# set relevant index for each row to 1 (one-hot encode)
#labels[np.arange(len(small_data)), arr] = 1
#print(labels)
#comment_labels=np.array(comment_labels)
#comment_texts=np.array(comment_texts)
#le = preprocessing.LabelEncoder()
#comment_labels=labels

#X_train, X_test, y_train, y_test = train_test_split(comment_texts, comment_labels, test_size=0.1, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(comment_texts_all, comment_labels_all, test_size=0.2, random_state=0)#0
#X_train2, X_test2, y_train2, y_test2 = train_test_split(comment_texts_all2, comment_labels_all2, test_size=0.1, random_state=0)

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
#tokenizer.set_truncation_and_padding(400,MAX_LENGTH,LONGEST_FIRST,0,0)
train_encodings = tokenizer(X_train,truncation=True,padding='max_length',max_length=400,return_tensors='np')#padding='max_length',max_length=512,
#train_encodings = tokenizer(comment_texts_all, truncation=True, padding=True,return_tensors='np')
#val_encodings = tokenizer(comment_texts2, truncation=True, padding=True, return_tensors='np')
test_encodings = tokenizer(X_test, truncation=True,padding='max_length',max_length=400,return_tensors='np')#padding='max_length',max_length=512,


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices(( # convert to dataset objects
    dict(train_encodings),
    (np.array(y_train))#.astype(int)#tf.convert_to_tensor
))
#val_dataset = tf.data.Dataset.from_tensor_slices((
#    dict(test_encodings),####################
#    np.array(y_test)
#))
#test_dataset = tf.data.Dataset.from_tensor_slices((
#    dict(val_encodings),
#    np.array(comment_labels2)
#))

#test_emb=tf.data.Dataset.from_tensor_slices(( # convert to dataset objects
#    dict(train_encodings)))

2022-01-10 12:18:18.196849: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-10 12:18:18.197363: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-10 12:18:18.264088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 12:18:18.264599: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-01-10 12:18:18.264651: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-10 12:18:18.265906: I tensorflow/stream_executor/platform/default/ds

In [29]:
train_dataset

<TensorSliceDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, ()), types: ({input_ids: tf.int64, attention_mask: tf.int64}, tf.int64)>

In [10]:
from transformers import TFAutoModelForSequenceClassification, TFTrainingArguments, TFTrainer
num_labels=4 #5 #4 #6 #3
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

2022-01-10 12:18:22.144212: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

Some layers of TFXLMRobertaForSequenceClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-large and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)#SparseCategoricalCrossentropy
optimizeradam = tf.keras.optimizers.Adam(learning_rate=0.000005)#0.000005

loss2=tf.keras.losses.BinaryCrossentropy(from_logits=True)

from tensorflow_addons.optimizers import AdamW
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [600], [5e-6, 1e-6])#-6#500#1500#700
   # lr and wd can be a function or a tensor
lr = 1 * schedule(step)
wd = lambda: 1e-6 * schedule(step)

schedule2=tf.optimizers.schedules.PolynomialDecay(5e-6,800,end_learning_rate=5e-7)#5e-7
lr2=1*schedule2(step)
wd2 = lambda: 1e-6 * schedule(step)

optimizer=AdamW(learning_rate=lr,weight_decay=wd)



model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])#model.compute_loss# can also use any keras loss fn, loss=model.compute_loss

In [55]:
#schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
#    [500], [5e-6, 1e-6])
   # lr and wd can be a function or a tensor
#lr = 1 * schedule(step)
#wd = lambda: 1e-6 * schedule(step)
#len(dict(train_encodings)['input_ids'])

In [12]:
his=model.fit(train_dataset.shuffle(1000).batch(8),epochs=5)#dict(train_encodings),np.array(y_train),epochs=5,batch_size=16 # #callbacks[history]#train_dataset.shuffle(1000).batch(16) #epochs=5,4,#validation_data=val_dataset.shuffle(1000).batch(16),callbacks=[es],dict(train_encodings),np.array(y_train).astype(np.float)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-01-10 12:18:37.640801: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-10 12:18:37.776218: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3600000000 Hz


1252/1252 [==============================] - 1188s 934ms/step - loss: 0.5854 - accuracy: 0.7493
Epoch 2/5
1252/1252 [==============================] - 1170s 935ms/step - loss: 0.0674 - accuracy: 0.9797
Epoch 3/5
1252/1252 [==============================] - 1170s 935ms/step - loss: 0.0454 - accuracy: 0.9875
Epoch 4/5
1252/1252 [==============================] - 1170s 934ms/step - loss: 0.0266 - accuracy: 0.9934
Epoch 5/5
1252/1252 [==============================] - 1170s 935ms/step - loss: 0.0209 - accuracy: 0.9950


In [13]:
out=model.predict(dict(test_encodings))#dict(val_encodings),dict(test_encodings),dat2,new_test_input
y_pred=np.array((np.argmax((tf.nn.softmax(out.logits,axis=-1)),axis=-1)))
print(classification_report(np.array(y_test),np.array(y_pred),digits=4))#comment_labels2,y_test
rep=classification_report(np.array(y_test),np.array(y_pred),digits=4)
#with open('result_4sent_2.txt','w+') as f:    

              precision    recall  f1-score   support

           0     0.9859    0.9813    0.9836       427
           1     0.9832    0.9936    0.9884       472
           2     0.9971    0.9943    0.9957       703
           3     0.9944    0.9933    0.9939       901

    accuracy                         0.9916      2503
   macro avg     0.9902    0.9906    0.9904      2503
weighted avg     0.9916    0.9916    0.9916      2503

